# YOLOv8 Training on Custom Dataset

## Create the Dataset

In [43]:
import yaml
import numpy as np
import pandas as pd
import os
import cv2

In [1]:
print('open the pod bay doors, hal')

open the pod bay doors, hal


### Get File Names of geom.yml and types.yml annotation files from train and validate folders
* `diva_train_path`
    - `train_files`
* `diva_valid_path`
    - `valid_files`

In [19]:
# get diva annotation files from training folder
diva_train_path = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/diva_annotations/train'
train_dir = os.listdir(diva_train_path)
keep_these = {".geom", "types"}
train_geom_types = sorted([item for item in train_dir if item[-9:-4] in keep_these])
train_files = list(zip(train_geom_types[::2], train_geom_types[1::2]))
train_files

[('VIRAT_S_000000.geom.yml', 'VIRAT_S_000000.types.yml'),
 ('VIRAT_S_000001.geom.yml', 'VIRAT_S_000001.types.yml'),
 ('VIRAT_S_000002.geom.yml', 'VIRAT_S_000002.types.yml'),
 ('VIRAT_S_000005.geom.yml', 'VIRAT_S_000005.types.yml'),
 ('VIRAT_S_000200_03_000657_000899.geom.yml',
  'VIRAT_S_000200_03_000657_000899.types.yml'),
 ('VIRAT_S_000200_05_001525_001575.geom.yml',
  'VIRAT_S_000200_05_001525_001575.types.yml'),
 ('VIRAT_S_000201_03_000640_000672.geom.yml',
  'VIRAT_S_000201_03_000640_000672.types.yml'),
 ('VIRAT_S_000201_05_001081_001215.geom.yml',
  'VIRAT_S_000201_05_001081_001215.types.yml'),
 ('VIRAT_S_000201_06_001354_001397.geom.yml',
  'VIRAT_S_000201_06_001354_001397.types.yml'),
 ('VIRAT_S_000201_07_001485_001581.geom.yml',
  'VIRAT_S_000201_07_001485_001581.types.yml'),
 ('VIRAT_S_000201_08_001652_001838.geom.yml',
  'VIRAT_S_000201_08_001652_001838.types.yml'),
 ('VIRAT_S_000202_02_001527_001560.geom.yml',
  'VIRAT_S_000202_02_001527_001560.types.yml'),
 ('VIRAT_S_00020

In [21]:
len(train_files)

64

In [20]:
# get diva annotation files from validation folder
diva_validate_path = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/diva_annotations/validate'
valid_dir = os.listdir(diva_validate_path)
keep_these = {".geom", "types"}
valid_geom_types = sorted([item for item in valid_dir if item[-9:-4] in keep_these])
valid_files = list(zip(valid_geom_types[::2], valid_geom_types[1::2]))
valid_files

[('VIRAT_S_000007.geom.yml', 'VIRAT_S_000007.types.yml'),
 ('VIRAT_S_000008.geom.yml', 'VIRAT_S_000008.types.yml'),
 ('VIRAT_S_000200_00_000100_000171.geom.yml',
  'VIRAT_S_000200_00_000100_000171.types.yml'),
 ('VIRAT_S_000200_02_000479_000635.geom.yml',
  'VIRAT_S_000200_02_000479_000635.types.yml'),
 ('VIRAT_S_000201_00_000018_000380.geom.yml',
  'VIRAT_S_000201_00_000018_000380.types.yml'),
 ('VIRAT_S_000201_01_000384_000589.geom.yml',
  'VIRAT_S_000201_01_000384_000589.types.yml'),
 ('VIRAT_S_000201_02_000590_000623.geom.yml',
  'VIRAT_S_000201_02_000590_000623.types.yml'),
 ('VIRAT_S_000201_04_000682_000822.geom.yml',
  'VIRAT_S_000201_04_000682_000822.types.yml'),
 ('VIRAT_S_000203_01_000171_000345.geom.yml',
  'VIRAT_S_000203_01_000171_000345.types.yml'),
 ('VIRAT_S_000203_08_001702_001734.geom.yml',
  'VIRAT_S_000203_08_001702_001734.types.yml'),
 ('VIRAT_S_000204_07_001577_001611.geom.yml',
  'VIRAT_S_000204_07_001577_001611.types.yml'),
 ('VIRAT_S_000204_09_001768_001849.geo

In [22]:
len(valid_files)

55

### Take a sample of the Train and Validate files for data extraction
- 30 from Train
- 10 from Validate

In [26]:
import random
random.seed(40)

train_sample = sorted(random.sample(train_files, 30))
valid_sample = sorted(random.sample(valid_files, 10))

In [29]:
len(train_sample)

30

In [30]:
len(valid_sample)

10

### Match Annotation Files to Video Files
* `train_sample` <--> `train_videos`
* `valid_sample` <--> `valid_videos`

In [34]:
train_videos = []

for i in range(len(train_sample)):
    this_file = train_sample[i][0].split('.')[0]
    this_file += '.mp4'
    train_videos.append(this_file)
len(train_videos)

30

In [36]:
print(train_sample[27][0])
print(train_videos[27])

VIRAT_S_050000_01_000207_000361.geom.yml
VIRAT_S_050000_01_000207_000361.mp4


In [35]:
valid_videos = []

for i in range(len(valid_sample)):
    this_file = valid_sample[i][0].split('.')[0]
    this_file += '.mp4'
    valid_videos.append(this_file)
len(valid_videos)

10

In [37]:
print(valid_sample[3][1])
print(valid_videos[3])

VIRAT_S_000206_08_001618_001712.types.yml
VIRAT_S_000206_08_001618_001712.mp4


### Grab the Track_ID of one person in the Training Set

In [46]:
train_sample[0]

('VIRAT_S_000001.geom.yml', 'VIRAT_S_000001.types.yml')

In [45]:
%%time
# geom
geom_path = diva_train_path + '/' + train_sample[0][0]
with open(geom_path, 'r') as file:
    geom = yaml.safe_load(file)

# types
types_path = diva_train_path + '/' + train_sample[0][1]
with open(types_path, 'r') as file:
    types = yaml.safe_load(file)

CPU times: user 55.5 s, sys: 243 ms, total: 55.7 s
Wall time: 55.7 s


In [50]:
# Turns the geom.yml and types.yml files into a single dataframe

detections = []  # all detections
id0 = []  # unique detection index
id1 = []  # track_id
ts0 = []  # frame
labels = []  # string class labels
labels_ints = []  # integer class labels
confs = []
xmin, ymin, xmax, ymax = [], [], [], []

for i in geom:
  try:
    if i['geom']['ts0'] is not None:  # populates detections data from geom file
      detections.append(i)
      id0.append(i['geom']['id0'])
      id1.append(i['geom']['id1'])
      ts0.append(i['geom']['ts0'])
      bb = i['geom']['g0'].split(' ')
      xmin.append(int(bb[0]))
      ymin.append(int(bb[1]))
      xmax.append(int(bb[2]))
      ymax.append(int(bb[3]))

      for j in types:
        try:
          if j['types']['id1'] == id1[-1]:  # pulls labels and confidences (by track id --> 'id1') from types.yaml file
            label, conf = next(iter(j['types']['cset3'].items()))
            confs.append(conf)
            labels.append(label)
            # also save labels as ints for comparison with YOLOv8 Predictions later on
            if label == 'Person':
              labels_ints.append(0)
            elif label == 'Bike':
              labels_ints.append(1)
            elif label == 'Vehicle':
              labels_ints.append(2)
            else:
              labels_ints.append(-1)
        except:
          pass
  except:
    pass

print('idx: ', len(idx),
      'track_id: ', len(track_id),
      'label: ', len(labels),
      'conf: ', len(confs),
      'frame: ', len(frame))  # sanity check


gt = {'idx_gt': id0, 'track_id_gt': id1, 'label_gt': labels,
      'label_as_int_gt': labels_ints, 'conf_gt': confs, 'frame_gt': ts0,
      'xmin_gt': xmin, 'ymin_gt': ymin, 'xmax_gt': xmax, 'ymax_gt': ymax}
df_gt = pd.DataFrame(gt)
df_gt

idx:  179573 track_id:  179573 label:  179573 conf:  179573 frame:  179573


,idx_gt,track_id_gt,label_gt,label_as_int_gt,conf_gt,frame_gt,xmin_gt,ymin_gt,xmax_gt,ymax_gt
0,0,1,Person,0,1.0,3455,1,663,77,795
1,1,1,Person,0,1.0,3456,1,663,77,795
2,2,1,Person,0,1.0,3457,1,663,77,795
3,3,1,Person,0,1.0,3458,1,663,77,795
4,4,1,Person,0,1.0,3459,1,663,77,795
...,...,...,...,...,...,...,...,...,...,...
179568,179568,5003,Other,-1,1.0,20650,760,758,778,830
179569,179569,5003,Other,-1,1.0,20651,760,758,778,830
179570,179570,5003,Other,-1,1.0,20652,760,758,778,830
179571,179571,5003,Other,-1,1.0,20653,760,758,778,830


In [58]:
# filter for cars and people
df_persons = df_gt[df_gt['label_as_int_gt'] == 0]
df_cars = df_gt[df_gt['label_as_int_gt'] == 2]

In [63]:
# sample frames: 30 cars and 70 persons
df_persons_70 = df_persons.sample(n=70)
df_cars_30 = df_cars.sample(n=30)

In [67]:
# get frame numbers from these samples
persons_frames = sorted(df_persons_70['frame_gt'].to_list())
cars_frames = sorted(df_cars_30['frame_gt'].to_list())

In [59]:
df_persons

,idx_gt,track_id_gt,label_gt,label_as_int_gt,conf_gt,frame_gt,xmin_gt,ymin_gt,xmax_gt,ymax_gt
0,0,1,Person,0,1.0,3455,1,663,77,795
1,1,1,Person,0,1.0,3456,1,663,77,795
2,2,1,Person,0,1.0,3457,1,663,77,795
3,3,1,Person,0,1.0,3458,1,663,77,795
4,4,1,Person,0,1.0,3459,1,663,77,795
...,...,...,...,...,...,...,...,...,...,...
95562,95562,25,Person,0,1.0,20651,1287,427,1335,547
95563,95563,25,Person,0,1.0,20652,1286,427,1334,547
95564,95564,25,Person,0,1.0,20653,1285,427,1333,547
95565,95565,25,Person,0,1.0,20654,1284,427,1332,547


In [60]:
df_cars

,idx_gt,track_id_gt,label_gt,label_as_int_gt,conf_gt,frame_gt,xmin_gt,ymin_gt,xmax_gt,ymax_gt
385,385,3,Vehicle,2,1.0,0,613,469,867,658
386,386,3,Vehicle,2,1.0,1,612,468,867,658
387,387,3,Vehicle,2,1.0,2,612,468,867,658
388,388,3,Vehicle,2,1.0,3,612,468,867,658
389,389,3,Vehicle,2,1.0,4,612,468,867,658
...,...,...,...,...,...,...,...,...,...,...
19276,19276,11,Vehicle,2,1.0,18684,1714,0,1873,39
19277,19277,11,Vehicle,2,1.0,18685,1717,0,1877,39
19278,19278,11,Vehicle,2,1.0,18686,1720,0,1880,39
19279,19279,11,Vehicle,2,1.0,18687,1723,0,1884,39
